### Start with importing the packages needed.

In [ ]:
import logging
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.filters import FunctionInvocationContext
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.chat_message_content import ChatMessageContent

# Load environment variables from .env file
load_dotenv()

# NOTE: This is all that is required to enable logging.
# Set the desired level to INFO, DEBUG, etc.
logging.basicConfig(level=logging.WARNING)

In [ ]:
# Define the auto function invocation filter that will be used by the kernel
async def function_invocation_filter(context: FunctionInvocationContext, next):
    """A filter that will be called for each function call in the response."""
    if "messages" not in context.arguments:
        await next(context)
        return
    #print(f"    Agent [{context.function.name}] called with messages: {context.arguments['messages']}")
    await next(context)
    #print(f"    Response from agent [{context.function.name}]: {context.result.value}")



In [ ]:
intermediate_steps: list[ChatMessageContent] = []

async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    intermediate_steps.append(message)

In [ ]:
# Create and configure the kernel.
from search import SearchPlugin

kernel = Kernel()

kernel.add_plugin(SearchPlugin(), plugin_name="search")

# The filter is used for demonstration purposes to show the function invocation.
kernel.add_filter("function_invocation", function_invocation_filter)

In [ ]:
# Create the Lead Market Analyst agent. NOTE: The name of the agent must not contain spaces.
lead_research_agent = ChatCompletionAgent(
    service= AzureChatCompletion(deployment_name="gpt-4o"),
    name="LeadResearchAgent",
    instructions=(
    """
    As the Lead Research Analyst at a premier content platform, you specialize in finding information that best fits the needs of your
    audience. Your primary responsibility is to ensure that the content produced by your team is not only accurate but also timely and
    relevant. You will do this by conducting in-depth research. You will need to search the web to get this content.
    """),
    plugins=[
        SearchPlugin()
    ],
    
)

In [ ]:
# Create the Data Analyst agent. NOTE: The name of the agent must not contain spaces.
data_analyst_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    name="DataAnalystAgent",
    instructions=(
    """
    As the Chief Data Strategist at a leading advisory firm, your expertise lies in analyzing vast datasets to uncover trends and
    opportunities that inform high-impact strategies with the goal of Synthesizing complex data into actionable insights 
    that can be transformed into compelling content.
    """),
)

In [ ]:
# Create the Content Creator agent. NOTE: The name of the agent must not contain spaces.
blog_creator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    name="BlogCreatorAgent",
    instructions=(
    """
    As the Content Creator at a leading advisory firm, your role is to transform complex data and insights into
    engaging and informative content that resonates with our audience. You will be responsible for creating articles, blogs, and other
    materials that effectively communicate our findings and recommendations. Additionally, you will collaborate with other agents to ensure
    the content aligns with our overall strategy and objectives.
    """),
)

In [ ]:
social_creator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    name="SocialCreatorAgent",
    instructions=(
    """
    As the Social Media Manager at a leading advisory firm, your role is to create engaging and impactful content for our social media
    platforms. You will be responsible for crafting tweets, posts, and other materials that effectively communicate our findings and
    recommendations. Any twitter posts, Instagram posts, and other posts should be very short and easy to understand, and should sound like clickbait.
    Additionally, you will collaborate with other agents to ensure the content aligns with our overall strategy and objectives.
    """),
)

In [ ]:
# Create the Quality Assurance agent. NOTE: The name of the agent must not contain spaces.
quality_assurance_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    name="QualityAssuranceAgent",
    instructions=(
    """
    As the Quality Assurance Specialist at a leading advisory firm, your role is to ensure that all content produced meets our
    high standards for accuracy, clarity, and relevance. You will review and edit content created by the team to ensure it aligns with our
    brand voice and effectively communicates our insights. Additionally, you will provide feedback to the social_creator_agent and the blog_creator_agent to enhance content quality.
    """),
)

In [ ]:
# Create the Coordinator agent. NOTE:The name of the agent must not contain spaces.
coordinator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    name="CoordinatorAgent",
    instructions=(
    """
    As the Coordinator at a leading advisory firm, your role is to facilitate communication and collaboration among team members,
    so that they can do research, write a blog, and create social media posts. You will be responsible for
    coordinating the efforts of the Lead Research Analyst, Data Analyst, Blog Creator, and Social Media Manager to ensure that all
    content is aligned with our overall strategy and objectives.
    
    """),
    plugins=[
        lead_research_agent,
        quality_assurance_agent,
        data_analyst_agent,
        blog_creator_agent,
        social_creator_agent
    ],
)

In [ ]:
thread: ChatHistoryAgentThread = None
#response = await coordinator_agent.get_response(
#    messages="Inflation in the US and the impact on the stock market in 2025",
#    thread=thread,
#    on_intermediate_message=handle_intermediate_steps,
#)

# Topic
topic = "Inflation in the US and the impact on the stock market in 2025"

async for response in coordinator_agent.invoke(messages=topic,
    thread=thread,
    on_intermediate_message=handle_intermediate_steps):
        #print(f"# Agent: {response}")
        thread = response.thread

# Print the intermediate steps
print("\nIntermediate Steps:")
for msg in intermediate_steps:
    if any(isinstance(item, FunctionResultContent) for item in msg.items):
        for fr in msg.items:
            if isinstance(fr, FunctionResultContent):
                print(f"\033[33mFunction Result for function: {fr.name} > \033[0m{fr.result} ")
    elif any(isinstance(item, FunctionCallContent) for item in msg.items):
        for fcc in msg.items:
            if isinstance(fcc, FunctionCallContent):
                print(f"\033[33mFunction Call:> {fcc.name} with arguments: {fcc.arguments}\033[0m")
    else:
        print(f"\033[34m{msg.role}: {msg.content}\033[0m")